ใน ep นี้เราจะมาเรียนรู้การสร้างแผนที่ แบบ Interactive ที่สามารถ Zoom เข้าออก และลากไปมา ได้เหมือน Google Map โดยใช้ folium 

# 0. Install

เราจะต้อง Install kaggle เพื่อ Download Dataset, geopandas เพื่อใช้ในการวิเคราะห์ข้อมูล geospatial, folium เพื่อแสดงแผ่นที่ (ถ้ายังไม่ได้ Install ให้ uncomment)

In [0]:
# ! pip install geopandas
# ! pip install git+https://github.com/python-visualization/folium
# ! pip install kaggle --upgrade

# 1. Import Library

Import folium Library เพื่อใช้ในการพล็อตแผนที่แบบ Interactive

In [0]:
import pandas as pd
import geopandas as gpd

import folium
from folium import *
from folium.plugins import *

import os
from pathlib import Path

ประกาศฟังก์ชันในการแสดงแผนที่โดยใช้ HTML iframe แต่ Colab ไม่ Support iframe เราจะ return Map ออกไปเลย

In [0]:
from IPython.display import IFrame, HTML

def embed_map(m, file_name):    
    # # VM
    # m.save(file_name)
    # return IFrame(src=file_name, width='100%', height='500px')

    # Colab
    return m

# 2. เตรียม Path สำหรับดาวน์โหลดข้อมูล

กำหนด path ของ Config File และ Dataset ว่าจะอยู่ใน Google Drive ถ้าเราใช้ Google Colab หรือ อยู่ใน HOME ถ้าเราใช้ VM ธรรมดา และกำหนด Environment Variable ไปยังโฟลเดอร์ที่เก็บ kaggle.json

ในกรณีใช้ Colab ให้ Mount Google Drive เพื่อดึง Config File มาจาก Google Drive ส่วนตัวของเรา เมื่อเรารัน Cell ด้านล่างจะมีลิงค์ปรากฎขึ้นมาให้เรา Login กด Approve แล้ว Copy Authorization Code มาใส่ในช่องด้านล่าง แล้วกด Enter

In [0]:
dataset = 'alexisbcook/geospatial-learn-course-data'

# Google Colab
config_path = Path('/content/drive')
data_path = Path('/content/datasets/')/dataset
from google.colab import drive
drive.mount(str(config_path))
os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/My Drive/.kaggle"

## VM
# config_path = Path(os.getenv("HOME"))
# data_path = config_path/"datasets"/dataset
# data_path.mkdir(parents=True, exist_ok=True)
# os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/.kaggle"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. Dataset

ในเคสนี้ เราจะสมมติตัวเองเป็นตำรวจในเมืองบอสตัน เราจะใช้ Dataset ที่เรามีมาวิเคราะห์อาชญากรรม ที่เกิดขึ้นในเมือง Boston ป้องกันปราบปรามอาชญากรรม ในเมืองนี้ให้ลดน้อยลง

สั่งดาวน์โหลด Dataset จาก Kaggle พร้อมทั้ง unzip ไว้ใน data_path

In [0]:
!kaggle datasets download {dataset} -p "{data_path}" --unzip

 97% 226M/233M [00:01<00:00, 164MB/s]
100% 233M/233M [00:01<00:00, 173MB/s]


# 4. พล็อตแผนที่ Interactive Map

เราจะลองพล้อตแผนที่ ที่ไหนก็ได้ในโลก ในที่นี้ เราจะใช้พิกัดของเมือง Boston และ zoom_start เริ่มต้นที่ 10 โดยใช้ tiles ของ openstreetmap 

* location คือ พิกัดจุดกึ่งกลางของแผนที่
* มี tiles รูปแบบอื่น ๆ อีกหลายอันให้เราได้เลือกใช้ เช่น openstreetmap, cartodbpositron, stamenterrain รายละเอียดเพิ่มเติมใน Credit ด้านล่าง
* zoom_start หมายถึง กำลังขยายเริ่มต้นที่จะแสดงแผนที่

In [0]:
# Create a map
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)

# Display the map
embed_map(m_1, '25d-m_1.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/10/25d-m_1.html" frameborder="0" allowfullscreen></iframe>

# 5. พล็อตจุดที่น่าสนใจ Plotting Points

## 5.1 เตรียมข้อมูล

เราจะใช้ Dataset อาชญากรรมในเมือง Boston

In [0]:
crimes = pd.read_csv(data_path/"crimes-in-boston/crimes-in-boston/crime.csv", encoding='latin')

crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


ตอนนี้เราจะสนใจเฉพาะ Feature Lat, Long ก่อน

In [0]:
crimes.describe()

,OFFENSE_CODE,YEAR,MONTH,HOUR,Lat,Long
count,319073.000000,319073.000000,319073.000000,319073.000000,299074.000000,299074.000000
mean,2317.546956,2016.560586,6.609719,13.118205,42.214381,-70.908272
std,1185.285543,0.996344,3.273691,6.294205,2.159766,3.493618
min,111.000000,2015.000000,1.000000,0.000000,-1.000000,-71.178674
25%,1001.000000,2016.000000,4.000000,9.000000,42.297442,-71.097135
50%,2907.000000,2017.000000,7.000000,14.000000,42.325538,-71.077524
75%,3201.000000,2017.000000,9.000000,18.000000,42.348624,-71.062467
max,3831.000000,2018.000000,12.000000,23.000000,42.395042,-1.000000


เนื่องจากข้อมูลค่อนข้างมาก เราจะ Focus เฉพาะ อาชญากรรมประเภท โจรกรรม ปล้น ชิงทรัพย์ ที่เกิดในเวลากลางวัน ในปี 2018 โดยเราจะ drop record ที่ไม่มีพิกัด Lat, Long ทิ้งไปก่อน

In [0]:
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crimes.YEAR == 2018) & \
                            (crimes.HOUR.isin(range(9,18))))].copy()

daytime_robberies.dropna(subset=['Lat', 'Long'], inplace=True)
daytime_robberies.describe()

,OFFENSE_CODE,YEAR,MONTH,HOUR,Lat,Long
count,232.000000,232.0,232.000000,232.000000,232.000000,232.000000
mean,324.405172,2018.0,4.370690,13.732759,42.322536,-71.077599
std,28.131564,0.0,2.335609,2.478855,0.031789,0.024994
min,301.000000,2018.0,1.000000,9.000000,42.241249,-71.159466
25%,301.000000,2018.0,2.000000,12.000000,42.296126,-71.090678
50%,311.000000,2018.0,4.000000,14.000000,42.326788,-71.074515
75%,361.000000,2018.0,6.000000,16.000000,42.350399,-71.061895
max,381.000000,2018.0,9.000000,17.000000,42.384902,-71.000495


## 5.2 Plot Marker

In [0]:
# Create a map
m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in daytime_robberies.iterrows():    
    Marker([row['Lat'], row['Long']]).add_to(m_2)

# Display the map
embed_map(m_2, '25d-m_2.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/10/25d-m_2.html" frameborder="0" allowfullscreen></iframe>

# 6. Marker Cluster

ถ้ามีจุดเป็นจำนวนมาก เวลาพล็อตรวมกันทีเดียวหมด จะทำให้ดูลำบาก เราสามารถใช้ Marker Cluster ในการรวมกลุ่ม Marker ที่อยู่ใกล้กันเข้าด้วยกัน โดยอัตโนมัติ เมื่อเรา Zoom เข้าไปถึงจะแยกออกเป็นแต่ละ Marker

In [0]:
# Create the map
m_3 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
mc = MarkerCluster()
for idx, row in daytime_robberies.iterrows():
    mc.add_child(Marker([row['Lat'], row['Long']]))
m_3.add_child(mc)

# Display the map
embed_map(m_3, '25d-m_3.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/10/25d-m_3.html" frameborder="0" allowfullscreen></iframe>

# 7. Bubble Map

Bubble Map ใช้วงกลมแทน Marker ทำให้เราสามารถกำหนดขนาด สีสัน เพื่อสื่อความหมาย แทนที่ความสัมพันธ์ของข้อมูล ตำแหน่งพิกัดภูมิศาสตร์ กับ อีก 2 Feature ตามที่เราต้องการ

ในเคสนี้ เราจะพล็อตวงกลมสีเขียวแทนการปล้นในช่วงเข้าถึงเที่ยง และวงกลมสีแดงในช่วงหลังเที่ยงถึงตอนเย็น โดยใช้ Circle

* location คือ ตำแหน่ง Lat, Long
* radius คือ รัศมีของวงกลม 
* color คือ สี ในเคสนี้เราจะเรียกไปยังฟังก์ชัน color_producer เพื่อคำนวนว่าจะใช้สีอะไร จากค่า HOUR

In [0]:
# Create a base map
m_4 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'

# Add a bubble map to the base map
for i in range(0,len(daytime_robberies)):
    Circle(
        location=[daytime_robberies.iloc[i]['Lat'], daytime_robberies.iloc[i]['Long']],
        radius=50,
        color=color_producer(daytime_robberies.iloc[i]['HOUR'])).add_to(m_4)

# Display the map
embed_map(m_4, '25d-m_4.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/10/25d-m_4.html" frameborder="0" allowfullscreen></iframe>

# 8. Heat Map

เราสามารถสร้าง แผนที่ความร้อน ที่เข้าใจง่าย ว่าบริเวณไหนเกิดเหตุการณ์บ่อย ความหนาแน่นมาก ก็จะเป็นสีแดง (ร้อน) เกิดน้อย ความหนาแน่นน้อย ก็จะเป็นสีน้ำเงิน (เย็น) ด้วย folium.plugins.HeatMap()

In [0]:
# crimes.dropna(subset=['Lat', 'Long'], inplace=True)

# Create a base map
m_5 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add a heatmap to the base map
HeatMap(data=daytime_robberies[['Lat', 'Long']], radius=16).add_to(m_5)

# Display the map
embed_map(m_5, '25d-m_5.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/10/25d-m_5.html" frameborder="0" allowfullscreen></iframe>

สังเกตว่าเมืองใหญ่ อาชญากรรมส่วนใหญ่มักจะเกิดบริเวณใจกลางเมือง

# Credit

* https://www.kaggle.com/alexisbcook/interactive-maps
* https://github.com/python-visualization/folium/tree/master/folium/templates/tiles
* https://python-visualization.github.io/folium/quickstart.html
* https://python-visualization.github.io/folium/plugins.html
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
* https://www.bualabs.com/archives/2717/plot-interactive-map-geospatial-analysis-japan-earthquake-tectonic-plate-boundary-folium-geospatial-ep-3/